In [4]:
import requests

# Get Wagtail Pages from Production

In [1]:
from urllib.parse import urljoin

In [13]:
base_api = "https://wersdoerfer.de/blogs/api/"
pages_endpoint = urljoin(base_api, "wagtail/pages/")
print(pages_endpoint)

https://wersdoerfer.de/blogs/api/wagtail/pages/


In [21]:
%%time
r = requests.get(pages_endpoint)
r.raise_for_status()
print(r.status_code)
data = r.json()

200
CPU times: user 22.5 ms, sys: 4.16 ms, total: 26.7 ms
Wall time: 151 ms


In [32]:
data["items"][2]

{'id': 5,
 'meta': {'type': 'cast.Post',
  'detail_url': 'https://wersdoerfer.de/blogs/api/wagtail/pages/5/',
  'html_url': 'https://wersdoerfer.de/blogs/das_claas_und_nora_blog/Zu-ersten-Mal-etwas-Mohrenbrei/',
  'slug': 'Zu-ersten-Mal-etwas-Mohrenbrei',
  'first_published_at': '2018-05-28T09:47:05.679581Z'},
 'title': 'Zu ersten Mal etwas Möhrenbrei'}

## Fetch Detail URLs

In [33]:
%%time
detail_urls = [(item["id"], item["meta"]["detail_url"]) for item in data["items"]]
details = {}
for item_id, detail_url in detail_urls:
    r = requests.get(detail_url)
    r.raise_for_status()
    details[item_id] = r.json()

CPU times: user 429 ms, sys: 53.8 ms, total: 482 ms
Wall time: 2.76 s


In [37]:
print(list(details.keys())[2])
details[5]

5


{'id': 5,
 'meta': {'type': 'cast.Post',
  'detail_url': 'https://wersdoerfer.de/blogs/api/wagtail/pages/5/',
  'html_url': 'https://wersdoerfer.de/blogs/das_claas_und_nora_blog/Zu-ersten-Mal-etwas-Mohrenbrei/',
  'slug': 'Zu-ersten-Mal-etwas-Mohrenbrei',
  'show_in_menus': False,
  'seo_title': '',
  'search_description': '',
  'first_published_at': '2018-05-28T09:47:05.679581Z',
  'alias_of': None,
  'parent': {'id': 3,
   'meta': {'type': 'cast.Blog',
    'detail_url': 'https://wersdoerfer.de/blogs/api/wagtail/pages/3/',
    'html_url': 'https://wersdoerfer.de/blogs/das_claas_und_nora_blog/'},
   'title': 'Claas und Nora sagen Hallo'}},
 'title': 'Zu ersten Mal etwas Möhrenbrei'}

In [38]:
r = requests.get("https://wersdoerfer.de/blogs/api/wagtail/pages/5/?fields=body")

In [45]:
r = requests.get("http://localhost:8000/cast/api/wagtail/pages/11/")
r.raise_for_status()

In [51]:
apidata = r.json()
apidata

{'id': 11,
 'meta': {'type': 'cast.Post',
  'detail_url': 'http://localhost:8000/cast/api/wagtail/pages/11/',
  'html_url': 'http://localhost:8000/mein-kleines-blog/audio-test/',
  'slug': 'audio-test',
  'show_in_menus': False,
  'seo_title': '',
  'search_description': '',
  'first_published_at': '2022-11-19T14:14:58.013575Z',
  'alias_of': None,
  'parent': {'id': 3,
   'meta': {'type': 'cast.Blog',
    'detail_url': 'http://localhost:8000/cast/api/wagtail/pages/3/',
    'html_url': 'http://localhost:8000/mein-kleines-blog/'},
   'title': 'Mein kleines Blog'}},
 'title': 'audio test',
 'uuid': '716212d8-2aea-4a85-b162-8b13a4908921',
 'visible_date': '2022-11-19T13:27:00Z',
 'comments_enabled': True,
 'body': [{'type': 'overview',
   'value': [{'type': 'heading',
     'value': 'Foo bar baz',
     'id': 'c75f42f0-bee5-4985-aeab-4912e3fb4c25'},
    {'type': 'video',
     'value': 1,
     'id': 'aed3e3a5-5d92-48d7-aad1-073698ed7c86'},
    {'type': 'audio',
     'value': 2,
     'id': 'd

In [55]:
exclude = {"meta"}
kwargs = {k: v for k, v in apidata.items() if k not in exclude}
Post(**kwargs)

<Post: audio test>

In [48]:
blog = Blog.objects.first()

In [60]:
def api_to_post(apidata):
    exclude = {"meta", "id"}
    kwargs = {k: v for k, v in apidata.items() if k not in exclude}
    return Post(**kwargs)

In [63]:
post = api_to_post(apidata)
post.slug = "audio-test-new"
blog.add_child(instance=post)

<Post: audio test>

In [65]:
post.pk

41

In [66]:
post.title = "audio test new 🤩"

In [67]:
post.save()

In [68]:
post.audios.all()

<AudioQuerySet [<Audio: 2 - pp 23 foobar>]>